## Importar librerias estandar

In [1]:
%matplotlib  inline
import cv2
import os
from os.path import join
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from tqdm import tqdm

## Cargar los modelos entrenados

In [2]:
from keras.models import load_model
### para el reconocimiento de digitos del dia y mes
model_cnn_recognized_digit=load_model("../models/model_cnn_recognized_digit.h5")
### para la clasificacion del año
model_svm_clasificacion_annio = pickle.load(open('../models/model_svm_clasificacion_annio.pkl', mode='rb'))

In [3]:
from keras.models import model_from_json
def load_model_custom(path):
    # carga el json y crea el modelo
    json_file = open(f'{path}.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # se cargan los pesos (weights) en el nuevo modelo
    loaded_model.load_weights(f'{path}.h5')
    return loaded_model
### para la identificacion del sign_1
model_vgg_identificacion_sign_1 = load_model_custom("../models/model_vgg_identificacion_sign_1")
### para la identificacion del sign_2
model_vgg_identificacion_sign_2 = load_model_custom("../models/model_vgg_identificacion_sign_2")
### para la identificacion del dia
model_vgg_identificacion_date_day = load_model_custom("../models/model_vgg_identificacion_date_day")
### para la identificacion del mes
model_vgg_identificacion_date_month = load_model_custom("../models/model_vgg_identificacion_date_month")
### para la identificacion de año
model_vgg_identificacion_date_year = load_model_custom("../models/model_vgg_identificacion_date_year")

In [4]:
### Definir el tamaño de la imagen
image_heigth = 56
image_width = 128

## Para la prediccion del resultado final, se ha optado por realizar 3 fases. 
* La primera fase trata de identificar las firmas y fechas, donde 1 indica la existencia del campo y 0 caso contrario.
* La segunda fase, trata de reconocer los digitos del dia y mes  de cada fecha
* La tercera fase, trata de clasificar si la imagen contiene el valor de 21 o 2021 para asi predecir el año

## Primera Fase

In [5]:
### definir la funcion para cargar la date de test
def load_dataset(ruta_data):
    data = []
    for i in tqdm(os.listdir(ruta_data)):
        if 'jpg' not in i:
            continue
        ruta_img = ruta_data + i
        image = cv2.imread(ruta_img)
        image = cv2.resize(image,(image_width,image_heigth))  
        data.append(image)
    data = np.array(data)
    return data

In [6]:
### definir la ruta del submission
ruta_submit = '../data/sampleSubmission.csv'

In [7]:
### crear diccionario por cada campo para asignar el nombre de columna y su modelo entrenado respectivamente
dict_target = {'sign_1':{'column':'firma1', 'model':model_vgg_identificacion_sign_1},
               'sign_2':{'column':'firma2', 'model':model_vgg_identificacion_sign_2},
               'date_day':{'column':'fecha', 'model':model_vgg_identificacion_date_day},
               'date_month':{'column':'fecha', 'model':model_vgg_identificacion_date_month},
               'date_year':{'column':'fecha', 'model':model_vgg_identificacion_date_year}}
### iterar por cada campo para predecir la existencia de sign_1, sign_2, date_day, date_month, date_year, donde "1" representa la existencia del campo
## y "0" caso contrario
list_y_test_column = []
for idx in dict_target:
    column =  dict_target[idx]['column']
    model =  dict_target[idx]['model']
    ### cargar la date  test de la imagen localizada por Yolo relacionado 
    ruta_data_test = f'../data/output/image_test_transform/{column}/'
    data_test = load_dataset(ruta_data_test)
    ### normalizar la data test
    data_test = data_test/255.0
    ### cargar la data de submission
    y_test_column = pd.read_csv(ruta_submit, keep_default_na = False, encoding = 'utf-8', dtype = 'str')
    ids = [ i.split('.')[0] for i in os.listdir(ruta_data_test) if 'jpg' in i]
    ### asegurarnos de que la data de submision este la alineada a las imagenes de la date test
    y_test_column = y_test_column.set_index('id').loc[ids]
    y_test_column = y_test_column[[idx]].copy()
    ### predecir el resultado
    pred = model.predict(data_test)
    ### definir el punto de corte, donde una probabilidad mayor o igual a "0.5" representa el valor de 1  y menor "0.5" representa el valor de 0
    pred[pred>=0.5] = 1
    pred[pred<0.5] = 0
    y_test_column[idx] = pred
    y_test_column[idx] = y_test_column[idx].astype(int)
    list_y_test_column.append(y_test_column)

100%|██████████| 108/108 [00:00<00:00, 3983.19it/s]


In [8]:
### obtener el resultado de la primera fase
df_submit = pd.concat(list_y_test_column, axis = 1)
df_submit = df_submit.replace({np.nan:1})
for i in df_submit.columns:
    df_submit[i] = df_submit[i].apply(lambda x: int(x))
df_submit.index.name = 'id'
df_submit = df_submit.reset_index()
df_submit.set_index('id', inplace=True)
df_submit

,sign_1,sign_2,date_day,date_month,date_year
id,,,,,
C_102,1,1,1,1,1
C_103,1,1,1,1,1
C_105,0,0,1,1,1
C_106,1,1,1,1,1
C_11,1,1,1,1,1
...,...,...,...,...,...
C_89,1,1,1,1,1
C_9,1,1,1,1,1
C_93,1,1,1,1,1


## Segunda Fase

In [9]:
### definir la funcion para identicar los picos altos de las imágenes
from scipy.signal import find_peaks
def get_peak(gray):
    mean_intensidad = np.mean(gray, axis=0)
    peaks, _ = find_peaks(mean_intensidad, height=0)
    _,w = gray.shape
    w = w//2
    idx = get_position(peaks, mean_intensidad[peaks], w)
    return idx 

In [10]:
## definir la funcion para identificar el punto de corte a dividir la imagen a nivel de columna
def get_position(peaks_posicion, peaks_intensidad, w):
    dict_value = dict(zip(peaks_posicion, peaks_intensidad))
    dict_value = {k: v for k, v in sorted(dict_value.items(), key=lambda item: item[1], reverse=True)}
    d = w
    for n, i in enumerate(dict_value):
        if np.abs(i-w)<d:
            d = np.abs(i-w)
            idx = i
    return idx

In [11]:
## definir la funcion para predecir el digito
def prediction(img):
    ## normalizar la imagen
    img = img/255.0
    ### redimensionae la imagen en un escala de 28x28
    img = cv2.resize(img, (28,28))
    ### hacer el reshape para que puede ser analizado por el modelo
    img = np.array(img).reshape(-1,28,28,1)
    ### predecir con el entrenado de reconocimiento de digitos
    pred_img = model_cnn_recognized_digit.predict(img)
    ### obtener el valo
    pred_img = np.argmax(pred_img,axis = 1) 
    return pred_img

In [12]:
def get_prediction_column(column, path):
    list_dict = []
    for i in os.listdir(path):
        img = cv2.imread(join(path,i)) 
        ### convertir a escala de grises
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ### calcular corte para dividir los numeros
        try:
            idx = get_peak(gray)    
        except:
            continue
        ## dividir la imagen en dos parte para asi obtener el digito del lado izquierdo y el digito del lado derecho
        izq = gray[:,:idx]
        der = gray[:,idx:]
        ### prediccion
        pred_izq = prediction(izq)
        pred_der = prediction(der)
#         print(pred_izq, pred_der)
        list_dict.append({'id':i.split('.')[0], f'{column}_izq':str(pred_izq[0]), f'{column}_der':str(pred_der[0])})
    return pd.DataFrame(list_dict)

In [13]:
### difinir la ruta donde se encuentar los numeros del dia y mes
dict_path = {'date_day':{'column':'day', 'path':'../data/output/image_test_transform/fecha/upscaling/digitos/day/'},
             'date_month':{'column':'month','path':'../data/output/image_test_transform/fecha/upscaling/digitos/month/'}}
list_df = []
for i in dict_path:
    df = get_prediction_column(dict_path[i]['column'], dict_path[i]['path'])
    list_df.append(df)

In [14]:
### juntar los de prediccion del dia y mes
df_day_month = pd.merge(list_df[0], list_df[1], how='inner', on='id')
### reemplazar el valor de 10 por un valor nulo para el mes
df_day_month['month_izq'] = df_day_month.month_izq.replace({'10':''})
df_day_month['month_der'] = df_day_month.month_der.replace({'10':''})
### unir los digitos de la izquierda y la derecha del campo date_month
df_day_month['date_month'] = df_day_month.month_izq + df_day_month.month_der
### reemplazar el valor de 10 por un valor nulo para el dia
df_day_month['day_izq'] = df_day_month.day_izq.replace({'10':''})
df_day_month['day_der'] = df_day_month.day_der.replace({'10':''})
### unir los digitos de la izquierda y la derecha del campo date_day
df_day_month['date_day'] = df_day_month.day_izq + df_day_month.day_der

In [15]:
### validar que cuando el valor sea mayor a 31 sea convertido a un volor de fecha real, en este caso se reemplazar el primer dígito por 1
mask = df_day_month['date_day'].astype(int)>31
df_day_month.loc[mask,'date_day'] = df_day_month[mask]['date_day'].apply(lambda x: '1'+x[1])
mask = df_day_month['date_month'].astype(int)>31
df_day_month.loc[mask,'date_month'] = df_day_month[mask]['date_month'].apply(lambda x: '1'+x[1])
column = ['id','date_day','date_month']
df_day_month = df_day_month[column]
df_day_month.set_index('id', inplace=True)

In [16]:
mask = df_submit['date_day'] == 0
###  juntar el resultado del primera fase con el de la segunda fase
df_submit = pd.concat([df_submit[['sign_1','sign_2','date_year']],df_day_month], axis=1)
### si es que existe valor "nan", reemplazarlo por el valor de moda de los campos predichos de date_day y date_month
df_submit['date_day'] = df_submit['date_day'].replace({np.nan:df_submit['date_day'].value_counts()[0]})
df_submit['date_month'] = df_submit['date_month'].replace({np.nan:df_submit['date_month'].value_counts()[0]})
### var que los valor valores 0 identificados en el primera fase se mantengan
df_submit.loc[mask,'date_day'] = 0
df_submit.loc[mask,'date_month'] = 0
column = ['sign_1','sign_2','date_day','date_month','date_year']
df_submit = df_submit[column]
df_submit

,sign_1,sign_2,date_day,date_month,date_year
id,,,,,
C_102,1,1,07,04,1
C_103,1,1,27,03,1
C_105,0,0,27,04,1
C_106,1,1,20,06,1
C_11,1,1,22,02,1
...,...,...,...,...,...
C_89,1,1,07,06,1
C_9,1,1,02,28,1
C_93,1,1,28,10,1


## Tercera Fase

In [17]:
### definir la ruta de imagenes de test de los años localizados por yolo
path = '../data/output/image_test_transform/fecha/upscaling/digitos/year/'
test = []
id_test = []
import os
for i in os.listdir(path):
    try:
        img = cv2.imread(path+str(i))
        img = img[:40,:]
        img = cv2.resize(img,(28,28))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        test.append(gray.ravel())
        id_test.append(i.split('.')[0])
    except Exception as e:
        print(e)
        pass
test = np.array(test)
test.shape

(101, 784)

In [18]:
## normalizar la data test
test = test/255.0
## predecir el resultado
pred = model_svm_clasificacion_annio.predict_proba(test)[:,-1]
## definir el punto de corte donde una probabilidad mayor a 0.4 representa 1 y caso contrario 0
pred[pred>0.4] = 1
pred[pred<=0.4] = 0

In [19]:
## reemplazar los valores predichos, mapenado 1 al año 21 y 0 al año 2021
df = pd.DataFrame(pred, index = id_test, columns =['date_day']).astype(int)
df= df.replace({1:'21',0:'2021'})

In [20]:
## reeemplaze esos valor predichos en la data que contiene los valores predicho de la primera y segunda fase
df_submit.loc[df.index,'date_year'] = df.values
df_submit = df_submit.reset_index()
df_submit

,id,sign_1,sign_2,date_day,date_month,date_year
0,C_102,1,1,07,04,2021
1,C_103,1,1,27,03,2021
2,C_105,0,0,27,04,2021
3,C_106,1,1,20,06,2021
4,C_11,1,1,22,02,2021
...,...,...,...,...,...,...
102,C_89,1,1,07,06,2021
103,C_9,1,1,02,28,2021
104,C_93,1,1,28,10,2021
105,C_96,1,1,12,06,2021


In [21]:
df_submit[df_submit.date_month==0]

,id,sign_1,sign_2,date_day,date_month,date_year
7,C_115,0,1,0,0,0
44,C_238,1,0,0,0,0
50,C_25,1,0,0,0,0
77,C_339,0,0,0,0,0
83,C_351,0,0,0,0,0
106,C_99,0,0,0,0,0


## Guardar el resultado final

In [22]:
df_submit.to_csv('../results/test_vgg_cnn_svm.csv', encoding = 'utf-8', sep = ',', index = False)